<a href="https://colab.research.google.com/github/caniro/GPT-Web/blob/main/191-%20Llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index html2text
!pip install llama-index-readers-web
!pip install openai
!pip install langchain
!pip install langchain-community

import os
from google.colab import userdata

# Colab Secret에서 API 키 불러옴
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader

# URL you want to load into your vector store here:
url = "http://www.paulgraham.com/fr.html"

# Load the URL into documents (multiple documents possible)
documents = SimpleWebPageReader(html_to_text=True).load_data([url])

# Create vector store from documents
index = VectorStoreIndex.from_documents(documents)

# Create query engine so we can ask it questions:
query_engine = index.as_query_engine()

# Ask as many questions as you want against the loaded data:
response = query_engine.query("What are the 3 best advise by Paul to raise money?")

In [8]:
print(response)

Avoid investors until you decide to raise money, talk to them all in parallel prioritized by expected value, and accept offers greedily.


LangChain


In [11]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory

loader = SimpleWebPageReader()
documents = loader.load_data(urls=[url])
index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [12]:
index.as_query_engine().query("What are the 3 best advise by Paul to raise money?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='1. Start by getting initial funding from sources like Y Combinator or individual angels, then gradually raise larger amounts as the company grows.\n2. Have multiple fundraising plans depending on how much you can raise, tailoring your pitch to different types of investors.\n3. Calculate the upper limit of what you should raise by multiplying the number of people you want to hire by $15k and 18 months, adding a margin for error.', source_nodes=[NodeWithScore(node=TextNode(id_='fe2503a0-a841-494d-901d-35b7c800caaf', embedding=None, metadata={'url': 'http://www.paulgraham.com/fr.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6e47154b-3557-43c6-a58a-a00e148ce656', node_type='4', metadata={'url': 'http://www.paulgraham.com/fr.html'}, hash='c4f1758462daa49e2fb4f4c5b9aac5a010302725e858bc9d9bf94ee88a2fe6a4'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='87982421-429a-4f

In [17]:
tools = [
    Tool(
        name="Website Index",
        func=lambda q: index.as_query_engine().query(q),
        description=f"Useful when you want answer questions about the text on websites.",
    ),
]

In [18]:
llm = OpenAI(temperature=1)
memory = ConversationBufferMemory(memory_key="chat_history")
agent_chain = initialize_agent(
    tools, llm, agent="zero-shot-react-description", memory=memory
)

output = agent_chain.run(input="What are the 3 best advise by Paul to raise money?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


In [19]:
output

'Know where you stand, get the first commitment, and close committed money'